In [66]:
import torch
import numpy as np
import pandas as pd
import random as rd
import csv

In [67]:
# default stuff
dtype = torch.double
device = torch.device("cpu")

# architecture definition
N_in, N_hid, N_out = 2, 10, 2
batch_size = 200
learning_rate = 0.001

#weight matrix
w1 = torch.randn(N_in, N_hid, device=device, dtype=dtype, requires_grad=True)
w2 = torch.randn(N_hid, N_out, device=device, dtype=dtype, requires_grad=True)

In [68]:
# data manipulation
# ho cambiato il file così ora sono 0 e 1 i colori
data = pd.read_csv('/home/burrito/Documents/python/pytorch/green_red/data_train.csv')
test = pd.read_csv('/home/burrito/Documents/python/pytorch/green_red/data_test.csv')
data_np_x = data[['x', 'y']].to_numpy()
data_np_y = data[['color']].to_numpy()
data_test = test.to_numpy()

In [69]:
batchesX = np.asarray(np.array_split(data_np_x, 100))
batchesY = np.asarray(np.array_split(data_np_y, 100))

In [70]:
Xtr = torch.from_numpy(batchesX[2])
Ytr = torch.from_numpy(batchesY[2])
Xtest = torch.from_numpy(data_test)

In [71]:
for epoch in range(100000):
    b = rd.randint(0, 99)
    Xtr = torch.from_numpy(batchesX[b])
    Ytr = torch.from_numpy(batchesY[b])
    y = Xtr.mm(w1).clamp(min=0).mm(w2)
    # ho provato a fare la cross-entropy ma non va, la 
    # la loss viene sempre uguale
    s = torch.gather(y, 1, Ytr) # gather serve a indicizzare y in base a Ytr
    loss = ((-((s.exp())/(y.exp().sum())).log())).sum()
    # questo valore qui sotto è a caso ma dovrebbe essere circa la lunghezza
    # di una batch
    loss = loss/100
    if epoch%10000 == 0:
        print(epoch, loss.item())
    loss.backward()
    with torch.no_grad():
        w1 -= learning_rate*w1.grad
        w2 -= learning_rate*w2.grad
        w1.grad.zero_()
        w2.grad.zero_()

0 5.403244883753766
10000 4.903159580926782
20000 4.832574211597049
30000 4.882154499869487
40000 4.746997983824702
50000 4.7802877797184
60000 4.776295445582914
70000 4.726057235307319
80000 4.716718953685786
90000 4.704618493001158


In [73]:
print(Xtest[0])
prediction = Xtest.mm(w1).clamp(min=0).mm(w2)

tensor([ 0.7026, -0.7593], dtype=torch.float64)


In [76]:
pred = open("/home/burrito/Documents/python/pytorch/green_red/pred.csv", 'w+', newline='')
writer = csv.writer(pred)
writer.writerow(["color"])
for i in prediction:
    if (i[0]<i[1]):
        writer.writerow(["1"])
    else:
        writer.writerow(["0"])